In [1]:
import pandas as pd
import requests
import openai
import os
import sys
import plotly.express as px
import json
from typing import List
from pydantic import BaseModel, Field
sys.path.append("../")
from dotenv import load_dotenv
from finance_functions import STOCK, get_ticker

In [2]:
# Load the environment variables from .env file
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [4]:
import langchain
print(langchain.__version__)

0.3.24


In [5]:
from langchain_core.tools import tool

In [6]:
get_ticker.invoke("Unicredit")

'UCG.MI'

In [8]:
from langchain_core.messages import HumanMessage

In [9]:
query = "What is the ticker for Unicredit?"

messages = [HumanMessage(query)]

In [10]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-nano-2025-04-14", model_provider="openai")

In [11]:
tools = [get_ticker]

In [12]:
llm_with_tools = llm.bind_tools(tools)

In [13]:
ai_msg = llm_with_tools.invoke(messages)

In [14]:
print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'get_ticker', 'args': {'company_name': 'Unicredit'}, 'id': 'call_w4IJpr0ccMZG0gMFh9eB2H5K', 'type': 'tool_call'}]


In [15]:
messages

[HumanMessage(content='What is the ticker for Unicredit?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w4IJpr0ccMZG0gMFh9eB2H5K', 'function': {'arguments': '{"company_name":"Unicredit"}', 'name': 'get_ticker'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQZ2U1IhysQUI5SCf7rHSSPh17rls', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f19a6811-b416-4a82-975a-dcc8b52f5d76-0', tool_calls=[{'name': 'get_ticker', 'args': {'company_name': 'Unicredit'}, 'id': 'call_w4IJpr0ccMZG0gMFh9eB2H5K', 'type': 'tool_call'}], usage_me

In [16]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_ticker": get_ticker}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is the ticker for Unicredit?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w4IJpr0ccMZG0gMFh9eB2H5K', 'function': {'arguments': '{"company_name":"Unicredit"}', 'name': 'get_ticker'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQZ2U1IhysQUI5SCf7rHSSPh17rls', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f19a6811-b416-4a82-975a-dcc8b52f5d76-0', tool_calls=[{'name': 'get_ticker', 'args': {'company_name': 'Unicredit'}, 'id': 'call_w4IJpr0ccMZG0gMFh9eB2H5K', 'type': 'tool_call'}], usage_me